## **title (Group Proposal)**

## Introduction

**Background**
(insert background info here)


**Question**
(insert question here)

**Dataset**
(identify and describe dataset here)

## Preliminary Data Analysis

Here we will show that our dataset can be read from the web into R. We will also wrangle our data into a tidy format and visualize it.

In [1]:
#read data here

In [2]:
#clean and wrangle data here

In [3]:
#using training data, summarize data into a table here

In [4]:
#using training data, visualize data here

## Methods

**Conducting Data Analysis**
(explain how we conduct data analysis here, and what variables/columns we will use)

**Visualizing results**
(describe a way we will visualize results)

## Expected Outcomes and Significance

**What do we expect to find?**
(answer here)

**What impact could our findings have?**
(answer here)

**What future questions could this lead to?**
(answer here)